In [48]:
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import random
import matplotlib.pylab as plt
import sklearn
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, Conv1D, MaxPool2D, MaxPooling2D
%matplotlib inline

In [49]:
imagePatches = glob('../input/breast-histopathology-images/IDC_regular_ps50_idx5/**/*.png', recursive=True)
for filename in imagePatches[0:10]:
    print(filename)

../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1851_y1251_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1451_y1551_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1101_y1301_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1301_y1251_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1651_y1101_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x2051_y1401_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x2051_y2401_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1601_y1751_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x1751_y1251_class1.png
../input/breast-histopathology-images/IDC_regular_ps50_idx5/12935/1/12935_idx5_x20

In [50]:
# Two arrays holding images by class type

class0 = [] # 0 = no cancer
class1 = [] # 1 = cancer

for filename in imagePatches:
    if filename.endswith("class0.png"):
         class0.append(filename)
    else:
        class1.append(filename)

In [66]:
sampled_class0 = random.sample(class0, 20000)
sampled_class1 = random.sample(class1, 20000)
len(sampled_class0)

20000

In [67]:
from matplotlib.image import imread
import cv2

def get_image_arrays(data, label):
    img_arrays = []
    for i in data:
      if i.endswith('.png'):
        img = cv2.imread(i ,cv2.IMREAD_GRAYSCALE)
        img_sized = cv2.resize(img, (50, 50), interpolation=cv2.INTER_LINEAR)
        img_arrays.append([img_sized, label])
    return img_arrays

In [68]:
class0_array = get_image_arrays(sampled_class0, 0)
class1_array = get_image_arrays(sampled_class1, 1)

In [69]:
test = cv2.imread('../input/breast-histopathology-images/IDC_regular_ps50_idx5/13689/1/13689_idx5_x801_y1501_class1.png' ,cv2.IMREAD_GRAYSCALE)
test.shape

(50, 50)

In [70]:
combined_data = np.concatenate((class0_array, class1_array))
random.seed(42)
random.shuffle(combined_data)

In [71]:
X = []
y = []

for features,label in combined_data:
    X.append(features)
    y.append(label)

In [72]:
# reshape X data
X = np.array(X).reshape(-1, 50, 50)
X.shape

(40000, 50, 50)

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(30000, 50, 50) (10000, 50, 50) (30000, 2) (10000, 2)


In [74]:
#create model
model_grayscale = Sequential()

#add model layers
model_grayscale.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(50,50)))
model_grayscale.add(Conv1D(32, kernel_size=3, activation='relu'))
model_grayscale.add(Flatten())
model_grayscale.add(Dense(2, activation='softmax'))

In [75]:
model_grayscale.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 48, 64)            9664      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 46, 32)            6176      
_________________________________________________________________
flatten_6 (Flatten)          (None, 1472)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 2946      
Total params: 18,786
Trainable params: 18,786
Non-trainable params: 0
_________________________________________________________________


In [76]:
#lesrning rsate for sgd rmsprop and adam, momentum 
model_grayscale.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
model_grayscale.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 30000 samples, validate on 10000 samples
Epoch 1/3
30000/30000 [==============================] - 7s 217us/step - loss: 1.4230 - accuracy: 0.8198 - val_loss: 0.4514 - val_accuracy: 0.8424
Epoch 2/3
30000/30000 [==============================] - 7s 229us/step - loss: 0.4338 - accuracy: 0.8464 - val_loss: 0.4337 - val_accuracy: 0.8435
Epoch 3/3
30000/30000 [==============================] - 7s 228us/step - loss: 0.4280 - accuracy: 0.8468 - val_loss: 0.4330 - val_accuracy: 0.8435


In [84]:
model_loss, model_accuracy = model_grayscale.evaluate(
    X_test, y_test, verbose=3)
print(f"Neural Network\n- Accuracy: {(model_accuracy*100):2f}%\n- Loss: {model_loss}")

Neural Network
- Loss: 0.43301055665016175
- Accuracy: 84.350002%
